# Mediator Recipe

### imports

In [ ]:
from aries_cloudcontroller import AriesAgentController
import os
import asyncio
from termcolor import colored

## Initialise Agent

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

In [ ]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

## Register Listeners

In [ ]:

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "checkRecords":
        # Your business logic
        print("check")
    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)




### Create Multi-Use Mediation Invitation

Note you could create single use invitations too, but likely as a mediator you would want the invitation to be multi-use

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(multi_use="true")
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

## Check Mediation Requests

In [ ]:
### To have a look at the mediation records and we should see our mediation in there
records_response = await agent_controller.mediation.get_mediation_records()
print(records_response)


## Grant Mediation Requests

If not handled automatically

In [ ]:
grant = await agent_controller.mediation.grant_mediation_request_by_id(records_response[0]["mediation_id"])
print(grant)